In [1]:
import findspark
findspark.init()
import pyspark ############ only run after findspark.init() ########
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.clustering import LDA
from pyspark.ml import Pipeline


import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import pandas as pd
import string
from tqdm import tqdm


import spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

%matplotlib inline

In [2]:
sc = SparkSession.builder.getOrCreate()
print(sc.version)

df_test = sc.sql('''select 'spark' as hello ''')
df_test.show()

2.3.2
+-----+
|hello|
+-----+
|spark|
+-----+



In [3]:
conn = sqlite3.connect("../../data/twitter.db")
df_bitcoin = pd.read_sql_query('select * from bitcoin where DateTime > "2018-03-03" and DateTime < "2018-05-03" and Language="en"', conn)
df_eth = pd.read_sql_query('select * from eth where DateTime > "2018-03-03" and DateTime < "2018-05-03" ', conn)
df_facebook = pd.read_sql_query('select * from facebook where DateTime > "2018-03-03" and DateTime < "2018-05-03" ', conn)


In [4]:
df_bitcoin.head()

,TweetID,ConversationID,AuthorId,AuthorName,isVerified,DateTime,Language,TweetText,Replies,Retweets,...,Permalink,URLs,isPartOfConversation,isReply,isRetweet,ReplyToUserID,ReplyToUserName,QuotedTweetID,QuotedTweetUserName,QuotedTweetUserID
0,991467272576397312,991467272576397312,2309334781,jgolivares9,0,2018-05-01 19:59,en,About damn time somebody realized it! economic...,0,0,...,/jgolivares9/status/991467272576397312,https://twitter.com/domcarter11/status/9914458...,0,0,1,,,991445821001433088,Domcarter11,321420513
1,991467247179743234,991467247179743234,377965799,BIECommunity,0,2018-05-01 19:59,en,@bethereumteam Many of you already correctly g...,0,0,...,/BIECommunity/status/991467247179743234,http://t.me/bethereum,0,0,0,,,,,
2,991467209527627776,991467209527627776,921421701681790977,NimbusToken,0,2018-05-01 19:59,en,Go check out the new @NimbusToken article on @...,4,99,...,/NimbusToken/status/991467209527627776,https://medium.com/@NimbusToken/nimbus-token-e...,0,0,0,,,,,
3,991467184768495618,991466590758518784,734191521457545218,Vaughn_Tron,0,2018-05-01 19:59,en,Fiat is dead #bitcoin,0,0,...,/Vaughn_Tron/status/991467184768495618,,1,1,0,169686021,@kanyewest,,,
4,991467183489171456,991467183489171456,724013469901590531,supercatmy,0,2018-05-01 19:59,en,Bitcoin - another major multi-resistance test ...,1,0,...,/supercatmy/status/991467183489171456,,0,0,0,,,,,


In [5]:
df_bitcoin = df_bitcoin.head(1000)
df_eth = df_eth.head(1000)
df_facebook = df_facebook.head(1000)

In [6]:
sqlContext = SQLContext(sc)

In [7]:
nlp = spacy.load('en_core_web_lg')

In [12]:
parser = English()
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
def spacy_tokenizer(sentence):
    print('in')
    mytokens = parser(sentence[1:100000])
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    #mytokens = " ".join([i for i in mytokens])
    return mytokens


In [13]:
spacy_tokenizer_udf = udf(f = spacy_tokenizer, returnType= ArrayType(StringType()))

In [14]:
df_bitcoin["processed_text"] = df_bitcoin["TweetText"].apply(spacy_tokenizer_udf)
df_eth["processed_text"] = df_bitcoin["TweetText"].apply(spacy_tokenizer_udf)
df_facebook["processed_text"] = df_bitcoin["TweetText"].apply(spacy_tokenizer_udf)

AnalysisException: 'syntax error in attribute name: #Bitcoin Killer is coming. Learn how #TIM is revolutionising bandwidth; security and utility on #blockchain. Download whitepaper: http://talking.im/tim.pdf Web: http://talking.im TG: http://t.me/timcommunity Join 4K+ members & Get going.. #ICO #ICOs #blockchainrevolution;'

In [16]:
spacy_tokenizer_udf = udf(f = spacy_tokenizer, returnType= ArrayType(StringType()))

> /home/apcloud44/spark-2.3.2-bin-hadoop2.7/python/pyspark/sql/utils.py(69)deco()
     67                                              e.java_exception.getStackTrace()))
     68             if s.startswith('org.apache.spark.sql.AnalysisException: '):
---> 69                 raise AnalysisException(s.split(': ', 1)[1], stackTrace)
     70             if s.startswith('org.apache.spark.sql.catalyst.analysis'):
     71                 raise AnalysisException(s.split(': ', 1)[1], stackTrace)



ipdb>  u


> /home/apcloud44/spark-2.3.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py(1257)__call__()
   1255         answer = self.gateway_client.send_command(command)
   1256         return_value = get_return_value(
-> 1257             answer, self.gateway_client, self.target_id, self.name)
   1258 
   1259         for temp_arg in temp_args:



ipdb>  answer


'xro102'


...

ipdb>  u


> /home/apcloud44/spark-2.3.2-bin-hadoop2.7/python/pyspark/sql/column.py(66)<listcomp>()
     64     """
     65     if converter:
---> 66         cols = [converter(c) for c in cols]
     67     return sc._jvm.PythonUtils.toSeq(cols)
     68 



ipdb>  cols


*** NameError: name 'cols' is not defined


ipdb>  q


In [10]:
tqdm.pandas()
df_bitcoin["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)
df_eth["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)
df_facebook["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)

100%|██████████| 1000/1000 [00:05<00:00, 117.13it/s]


In [11]:
df_facebook.head()

,TweetID,ConversationID,AuthorId,Author Name,isVerified,DateTime,TweetText,Replies,Retweets,Favorites,...,URLs,isPartOfConversation,isReply,isRetweet,ReplyToUserID,ReplyToUserName,QuotedTweetID,QuotedTweetUserName,QuotedTweetUserID,processed_text
0,991887071487946754,991887071487946754,737333614518620161,DigitalLeadersA,0,2018-05-02 23:48,Over The Internet No One Can ‘Hear’ you scream...,0,3,2,...,https://buff.ly/2jpflKI,0,0,0,,,,,,"[bout, damn, time, somebody, realize, economic..."
1,991886944123600896,991760296841752583,606152349,Dawniewouldnt,0,2018-05-02 23:47,#Emerdata #Emerdata is #CambridgeAnalytica #Ca...,0,1,1,...,,1,0,0,,,,,,"[bethereumteam, correctly, guess, come, day, h..."
2,991886869850976257,991886869850976257,1326825139,CharlotteKent20,0,2018-05-02 23:47,Do you follow our #CaptainsPointStories #blog ...,0,2,1,...,https://annieacornpublishing.com/promotions/ca...,0,0,0,,,,,,"[o, check, new, @nimbustoken, article, @medium..."
3,991886806554570752,991886806554570752,2304720642,vikramwkarve,0,2018-05-02 23:46,Retweeted Jagrati Shukla ( @JagratiShukla29): ...,1,0,1,...,,0,0,0,,,,,,"[iat, dead, bitcoin]"
4,991886783783686145,991886783783686145,1293576662,kevyna30,0,2018-05-02 23:46,just log out #Facebook for tonight,0,0,0,...,,0,0,0,,,,,,"[itcoin, major, multi, resistance, test, weekl..."


In [12]:
cv = CountVectorizer(inputCol="processed_text", outputCol="rawFeatures", vocabSize=1000, minDF=2.0)
df_spark_bitcoin = sqlContext.createDataFrame(df_bitcoin.loc[:,['TweetID','processed_text']])
df_spark_eth = sqlContext.createDataFrame(df_eth.loc[:,['TweetID','processed_text']])
df_spark_facebook = sqlContext.createDataFrame(df_facebook.loc[:,['TweetID','processed_text']])

model = cv.fit(df_spark_bitcoin)

In [13]:
df_spark_bitcoin_cv = model.transform(df_spark_bitcoin)
df_spark_eth_cv = model.transform(df_spark_eth)
df_spark_facebook_cv = model.transform(df_spark_facebook)

In [14]:
df_spark_bitcoin_cv.head()

Row(TweetID='991467272576397312', processed_text=['bout', 'damn', 'time', 'somebody', 'realize', 'economic', 'power', 'ultimate', 'power', 'freedom', 'blockchain', 'buildalegacy', 'bitcoin', 'cryptohttps://twitter.com', 'domcarter11/status/991445821001433088', '…'], rawFeatures=SparseVector(1000, {0: 1.0, 5: 1.0, 8: 1.0, 85: 1.0, 503: 2.0}))

In [15]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(df_spark_bitcoin_cv)
df_spark_bitcoin_idf = idfModel.transform(df_spark_bitcoin_cv)

In [16]:
df_spark_bitcoin_idf.head()

Row(TweetID='991467272576397312', processed_text=['bout', 'damn', 'time', 'somebody', 'realize', 'economic', 'power', 'ultimate', 'power', 'freedom', 'blockchain', 'buildalegacy', 'bitcoin', 'cryptohttps://twitter.com', 'domcarter11/status/991445821001433088', '…'], rawFeatures=SparseVector(1000, {0: 1.0, 5: 1.0, 8: 1.0, 85: 1.0, 503: 2.0}), features=SparseVector(1000, {0: 0.003, 5: 1.1342, 8: 1.1219, 85: 3.2452, 503: 8.4014}))

In [17]:
lda = LDA(featuresCol='features', k=25, seed=1, optimizer="em")

#pipeline = Pipeline(stages=[idf, lda])

#ldamodel = lda.fit(df_spark_bitcoin_idf)
#df_spark_bitcoin_lda = ldamodel.transform(df_spark_bitcoin_idf)

In [18]:
#df_spark_bitcoin_lda.head()

In [19]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer as sk_cv
from sklearn.manifold import TSNE
import pyLDAvis.sklearn
def spacy_tokenizer(sentence):
    mytokens = parser(sentence[1:100000])
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

df_bitcoin["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)
df_eth["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)
df_facebook["processed_text"] = df_bitcoin["TweetText"].progress_apply(spacy_tokenizer)

100%|██████████| 1000/1000 [00:05<00:00, 113.67it/s]


In [20]:
df_all = pd.concat([df_bitcoin, df_eth, df_facebook], sort=False)

In [21]:
vectorizer = sk_cv(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df_all.loc[:,'processed_text'])

<>:1: DeprecationWarning: invalid escape sequence \-
<>:1: DeprecationWarning: invalid escape sequence \-
<>:1: DeprecationWarning: invalid escape sequence \-
<ipython-input-21-50e7bd3aba27>:1: DeprecationWarning: invalid escape sequence \-
  vectorizer = sk_cv(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [22]:
NUM_TOPICS = 10
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [23]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [24]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('com', 367.52057122413515), ('twitter', 273.1601137768893), ('blockchain', 264.8840657836899), ('https', 257.4451792290491), ('http', 204.30997964835245), ('pic', 148.867450614949), ('cash', 138.4271998020571), ('btc', 135.67407782809704), ('ethereum', 134.51682242850165), ('exchange', 119.15575794271982)]
Topic 1:
[('com', 448.22005228492446), ('https', 323.7760589492861), ('twitter', 259.60161437469804), ('cryptocurrency', 257.5929613804996), ('crypto', 243.5716476969917), ('http', 192.81704041339245), ('btc', 170.7312422123826), ('blockchain', 169.07517933953892), ('new', 109.12993892341993), ('trading', 107.28307388613689)]
Topic 2:
[('btc', 509.83573556290423), ('com', 332.17921446461656), ('price', 247.41227263465026), ('usd', 233.36525318755503), ('twitter', 183.0360235524321), ('https', 180.88736447775074), ('monero', 174.71474979419114), ('crypto', 171.0832749643402), ('cryptocurrency', 169.22952814814056), ('eth', 152.75499734764966)]
Topic 3:
[('com', 2

In [25]:
df_eth.TweetText[0]

"RT _JohnWhelan: Delighted to have been a co-chair of the EEA's Technical Steering Committee that has been behind this effort. A big thanks to all involved. https://www.coindesk.com/enterprise-ethereum-alliance-isnt-dead-got-roadmap-prove/ … EntEthAlliance #ethereum #crypto #cryptocurrency"

In [26]:
text = spacy_tokenizer("RT _JohnWhelan: Delighted to have been a co-chair of the EEA's Technical Steering Committee that has been behind this effort. A big thanks to all involved. https://www.coindesk.com/enterprise-ethereum-alliance-isnt-dead-got-roadmap-prove/ … EntEthAlliance #ethereum #crypto #cryptocurrency")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.00714419 0.42366687 0.0071458  0.12573126 0.00714531 0.00714557
 0.00714567 0.08851834 0.31921034 0.00714667]


In [27]:
pyLDAvis.enable_notebook()

In [28]:
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

/home/apcloud44/miniconda3/envs/p3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5     -359.039581   38.244698       1        1  58.902033
9      223.615585 -101.886688       2        1  10.809021
8      220.939301  134.484787       3        1   9.738971
1       18.968201  -69.574844       4        1   4.390511
0       12.634027 -274.803894       5        1   3.750874
2     -211.508438 -185.146469       6        1   3.581749
6       11.519470  129.864929       7        1   2.700864
3     -157.491058   23.812206       8        1   2.594503
7     -199.942429  244.086594       9        1   2.241196
4       32.893166  332.235535      10        1   1.290277, topic_info=     Category         Freq            Term        Total  loglift  logprob
779   Default  9296.000000             com  9296.000000  30.0000  30.0000
550   Default  5720.000000             btc  5720.000000  29.0000  29.0000
1755  Default  8847.000000           https  8847.000000  28.0000  28.0000
903   Default  6643.000000          crypto  6643.000000  27.0000  27.0000
3436  Default  5417.000000         twitter  5417.000000  26.0000  26.0000
1754  Default  4365.000000            http  4365.000000  25.0000  25.0000
462   Default  5677.000000      blockchain  5677.000000  24.0000  24.0000
927   Default  5511.000000  cryptocurrency  5511.000000  23.0000  23.0000
3143  Default  9245.000000          status  9245.000000  22.0000  22.0000
1314  Default  5160.000000        ethereum  5160.000000  21.0000  21.0000
2390  Default  1825.000000            news  1825.000000  20.0000  20.0000
3665  Default  2268.000000             www  2268.000000  19.0000  19.0000
1776  Default  3056.000000             ico  3056.000000  18.0000  18.0000
1311  Default  2179.000000             eth  2179.000000  17.0000  17.0000
2597  Default  1360.000000             pic  1360.000000  16.0000  16.0000
604   Default   478.000000             buy   478.000000  15.0000  15.0000
349   Default  1243.000000             bit  1243.000000  14.0000  14.0000
2676  Default   948.000000           price   948.000000  13.0000  13.0000
368   Default   879.000000     bitcoincash   879.000000  12.0000  12.0000
3485  Default   559.000000             usd   559.000000  11.0000  11.0000
3370  Default   827.000000           trade   827.000000  10.0000  10.0000
2106  Default  1103.000000        litecoin  1103.000000   9.0000   9.0000
635   Default   517.000000           cards   517.000000   8.0000   8.0000
1347  Default   655.000000        exchange   655.000000   7.0000   7.0000
1797  Default  1059.000000             ift  1059.000000   6.0000   6.0000
2195  Default   558.000000          market   558.000000   5.0000   5.0000
3341  Default  1069.000000           token  1069.000000   4.0000   4.0000
2381  Default   328.000000             net   328.000000   3.0000   3.0000
3344  Default  1050.000000       tokensale  1050.000000   2.0000   2.0000
2147  Default   698.000000             ltc   698.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
2979  Topic10    99.573529            sell   195.665036   3.6748  -3.3867
604   Topic10   152.048403             buy   478.882020   3.2031  -2.9634
1844  Topic10    57.185649            info   160.194091   3.3202  -3.9413
2001  Topic10    19.410783         koineks    38.009622   3.6783  -5.0218
2131  Topic10    15.744757           looks    29.150070   3.7344  -5.2311
1660  Topic10    15.238251          happen    31.587912   3.6213  -5.2638
513   Topic10     9.777692             bpi    16.284195   3.8402  -5.7075
2551  Topic10     7.792583         passive    11.758003   3.9390  -5.9345
1903  Topic10    19.442921       investors    56.799777   3.2783  -5.0202
3706  Topic10    31.441403            year   142.958952   2.8359  -4.5395
3560  Topic10    32.219346          volume   156.754875   2.7682  -4.5151
2143  Topic10    24.829981             low    98.873709   2.9685  -4.7756
1754 

Factors from LDA

In [29]:
data_lda

array([[6.66751808e-03, 6.66825839e-03, 6.66715826e-03, ...,
        6.66700058e-03, 6.66813778e-03, 6.66713235e-03],
       [9.49991478e-01, 5.55628927e-03, 5.55687695e-03, ...,
        5.55679577e-03, 5.55685378e-03, 5.55581958e-03],
       [5.88304622e-03, 5.88423892e-03, 5.88323264e-03, ...,
        5.88249535e-03, 5.88380891e-03, 5.88247164e-03],
       ...,
       [9.09266441e-03, 9.09223974e-03, 9.09264478e-03, ...,
        1.61143232e-01, 9.09195272e-03, 1.32374407e-01],
       [1.68957466e-04, 1.68959472e-04, 2.70677769e-02, ...,
        1.68939338e-04, 4.26290884e-01, 1.70372859e-01],
       [9.09272447e-03, 9.09305070e-03, 9.09206682e-03, ...,
        1.37764673e-01, 9.09201874e-03, 9.09176993e-03]])

In [30]:
data_lda.shape

(3000, 10)

In [31]:
svd_2d = TruncatedSVD(n_components=2)
data_2d = svd_2d.fit_transform(data_vectorized)

In [40]:
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

init_notebook_mode(connected=True)


trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names(),
    hovertext = vectorizer.get_feature_names(),
    hoverinfo = 'text' 
)
data = [trace]
iplot(data, filename='scatter-mode')

# Values after analysis

## LSI

U matrix

In [33]:
data_2d[1:10]

array([[ 1.33647623e+00, -2.50250684e-01],
       [ 2.11720450e+00, -3.55493013e-01],
       [ 6.78186982e-04,  4.58770008e-03],
       [ 6.28749965e-01,  1.35965922e-01],
       [ 9.75271775e-01,  3.61376372e-01],
       [ 1.13029995e+00,  3.82572851e-01],
       [ 6.81727909e-01,  2.53008707e-01],
       [ 6.88884561e-01,  1.51602835e-01],
       [ 2.47757613e+00,  1.26321373e-01]])

Sigma matrix

In [34]:
svd_2d.singular_values_

array([3302.41389046,  427.61861598])

V Matrix

In [35]:
svd_2d.components_

array([[ 0.00076947,  0.00053112,  0.00072803, ...,  0.00050372,
         0.00033374,  0.00068757],
       [-0.00185975, -0.00233044, -0.00559844, ...,  0.00294034,
        -0.00103076,  0.00109277]])

# Dimensions

Actual Data

In [36]:
data_vectorized.shape

(3000, 3746)

U Matrix

In [37]:
data_2d.shape

(3000, 2)

Sigma Matrix

In [38]:
svd_2d.singular_values_.shape

(2,)

V Matrix

In [39]:
svd_2d.components_.shape

(2, 3746)